In [20]:
import argparse
import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms


In [57]:
# Open High Low Close
train_data=pd.read_csv('training.csv',header=None)

# print(train_data[0])
# train_data.head()
# train_data.shape

0       186.73
1       185.57
2       184.81
3       184.39
4       182.20
         ...  
1483    151.95
1484    152.06
1485    152.35
1486    152.81
1487    153.65
Name: 0, Length: 1488, dtype: float64


In [76]:
def get_data(df, days_to_train=3, days_to_pred=1):
    # 用前三天預測當天
    X=[]
    Y=[]

    df=df.apply(lambda x:(x-x.mean())/ x.std())
    
    for i in range(df.shape[0]-days_to_train):
        X.append(np.array(df.iloc[i:(i+days_to_train)]))
        Y.append(np.array(df.iloc[i+days_to_train,0]))

    return np.array(X), np.array(Y)

class DataSet(Dataset):

    def __init__(self,a, b, trans):
        self.train=a
        self.valid=b
        self.transform=trans

    def __getitem__(self, index) :
        x=self.train[index]
        y=self.valid[index]

        x=self.transform(x)
        # y=self.transform(y)

        return x, y

    def __len__(self):

        return len(self.train)

In [79]:
X, Y=get_data(train_data)
# print(X)

train_x, train_y=X[:int(len(X)*0.8)], Y[:int(len(Y)*0.8)]

# transform=transforms.Compose([
#     transforms.ToTensor(),
# ])
# train_loader=DataLoader(dataset=DataSet(train_x, train_y, transform),batch_size=12,shuffle=True)

# print(X[0],Y[1])

array([[0.62957606, 0.66460774, 0.64639977, 0.60699286],
       [0.57652336, 0.5561619 , 0.59786149, 0.57227652],
       [0.54176469, 0.49692677, 0.51360636, 0.53207866]])